In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Kitchen_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37000337|R3DT59XH7HXR9K|B00303FI0G|     529320574|Arthur Court Pape...|         Kitchen|          5|            0|          0|   N|                Y|Beautiful. Looks ...|Beautiful.  Looks...| 2015-08-31|
|         US|   15272914|R1LFS11BNASSU8|B00JCZKZN6|     274237558|Olde Thompson Bav...|         Kitchen|          5|    

In [12]:
# ReCreate the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3DT59XH7HXR9K|          5|            0|          0|   N|                Y|
|R1LFS11BNASSU8|          5|            0|          1|   N|                Y|
|R296RT05AG0AF6|          5|            0|          0|   N|                Y|
|R3V37XDZ7ZCI3L|          5|            0|          1|   N|                Y|
|R14GU232NQFYX2|          5|            0|          0|   N|                Y|
| RZQH4V7L2O1PL|          1|            1|          1|   N|                Y|
|R1F8JMOSPJ3KO7|          5|            0|          0|   N|                Y|
|R1ZISGY2BWW4Z5|          5|            0|          0|   N|                Y|
|R17PW4I3AE5WZW|          5|            0|          0|   N|                Y|
|R3D93G1KTP6A8P|          3|            0|          0|   N|     

In [13]:
#1 Filter for total_votes count is greater than or equal to 20
total_votes_df = vine_df.filter(vine_df.total_votes>=20)
total_votes_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R28RB82UG4RDD5|          5|           20|         20|   N|                Y|
|R3FB6BERWPEIJP|          4|           40|         43|   N|                Y|
|R1D4Z38STRDQXK|          5|           53|         56|   N|                Y|
|R1XMWJZICINIFX|          3|           20|         21|   N|                Y|
|R20QKY1GABXFLM|          1|          272|        297|   N|                Y|
|R3GZ7CK2O1PPG0|          2|           10|         30|   N|                Y|
|R328FA1E6FY3F5|          5|           17|         20|   N|                N|
|R3DH22AA5WGLLS|          5|           30|         30|   N|                N|
|R1E0E5EFZSLJCS|          1|           66|         80|   N|                Y|
|R1TXZQWEHYWEWN|          2|           48|         51|   N|     

In [14]:
#2 Filter for helpful votes/total votes is equal to or greater thant 50%
helpful_votes_df = vine_df.filter(vine_df.helpful_votes/vine_df.total_votes >= 0.5)
helpful_votes_df.show(10)


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RZQH4V7L2O1PL|          1|            1|          1|   N|                Y|
|R2ZD1IGC9UU55X|          5|            1|          1|   N|                Y|
|R28RB82UG4RDD5|          5|           20|         20|   N|                Y|
|R25A9174CPVZ99|          4|            2|          2|   N|                Y|
|R2S0LEB4UYZBHL|          5|            1|          1|   N|                Y|
| RB3VI0SCCHUKL|          1|            1|          1|   N|                Y|
|R3DT3FFRVZ32WC|          1|            5|          5|   N|                Y|
|  RIH54UBHI039|          4|            1|          1|   N|                Y|
|R2QH363VPMZD1V|          5|            1|          2|   N|                Y|
|R2SRBLOANDGWI5|          5|            1|          1|   N|     

In [17]:
#3 Filter for reviews with Vine Review
reviews_paid_df = helpful_votes_df.filter(helpful_votes_df.vine == 'Y')
reviews_paid_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1Z5WEY1AZVTC3|          3|            6|          6|   Y|                N|
|R3HGKU6KKKUC7B|          4|            1|          1|   Y|                N|
| RHS9A37CFQN7Q|          5|            1|          1|   Y|                N|
|R1237PYBSQBKQT|          5|            1|          2|   Y|                N|
|R24OIMAB8K6JWI|          5|            1|          2|   Y|                N|
|R235259QWFPB33|          4|            6|          9|   Y|                N|
| RM451PS0M43RG|          5|            1|          2|   Y|                N|
|R1NOKNNSDJB7WO|          4|            1|          1|   Y|                N|
| RTCKDNLR4FSVQ|          5|            2|          2|   Y|                N|
|R39MJC5LN6GJHM|          4|            1|          1|   Y|     

In [18]:
#4 Filter for written reviews that were not part of the Vine program
reviews_unpaid_df = helpful_votes_df.filter(helpful_votes_df.vine == 'N')
reviews_unpaid_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RZQH4V7L2O1PL|          1|            1|          1|   N|                Y|
|R2ZD1IGC9UU55X|          5|            1|          1|   N|                Y|
|R28RB82UG4RDD5|          5|           20|         20|   N|                Y|
|R25A9174CPVZ99|          4|            2|          2|   N|                Y|
|R2S0LEB4UYZBHL|          5|            1|          1|   N|                Y|
| RB3VI0SCCHUKL|          1|            1|          1|   N|                Y|
|R3DT3FFRVZ32WC|          1|            5|          5|   N|                Y|
|  RIH54UBHI039|          4|            1|          1|   N|                Y|
|R2QH363VPMZD1V|          5|            1|          2|   N|                Y|
|R2SRBLOANDGWI5|          5|            1|          1|   N|     

In [22]:
#5 Determine the total number of reviews, the number of 5-star reviews and the percentage of 5 star reviews for the two types of review, paid vs unpaid
total_paid_reviews = reviews_paid_df.count()
total_paid_reviews


12228

In [25]:
paid_5_star_reviews = reviews_paid_df.filter(reviews_paid_df.star_rating == 5).count()
paid_5_star_reviews

5667

In [26]:
# percentage of 5 star reviews for paid reviews
percent_paid = (paid_5_star_reviews/total_paid_reviews) * 100
percent_paid


46.34445534838076

In [27]:
# total unpaid reviews
total_unpaid_reviews = reviews_unpaid_df.count()
total_unpaid_reviews

1617533

In [28]:
# number of unpaid 5 star reviews
unpaid_5_star_reviews = reviews_unpaid_df.filter(reviews_written_df.star_rating ==5).count()
unpaid_5_star_reviews

913077

In [29]:
# percentage of 5 star reviews, upaid
percent_unpaid = (unpaid_5_star_reviews/total_unpaid_reviews) * 100
percent_unpaid

56.4487401493509